In [242]:
#!pip install pyspark

In [243]:
import pyspark

In [244]:
import pandas as pd 
import csv

list = [['samaha','IOT','1000'],['imane','Big data','2000'],['adam','data science','8'],
['mohamed','IOT','9000'],['ahmed','Big data','4000'],['adam','data science','5679'],
['','','6'],['','37','']

]
with open(r'C:\Users\samah\Desktop\test.csv', 'w' , newline = '') as f:
    thewriter = csv.writer(f)
    thewriter.writerow(['name','departement','experience'])
    for l in list:
        thewriter.writerow(l)


In [245]:
pd.read_csv(r'C:\Users\samah\Desktop\test.csv')

,name,departement,experience
0,samaha,IOT,1000.0
1,imane,Big data,2000.0
2,adam,data science,8.0
3,mohamed,IOT,9000.0
4,ahmed,Big data,4000.0
5,adam,data science,5679.0
6,NaN,NaN,6.0
7,NaN,37,NaN


In [246]:
# spark sessions in one cluster 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practice').getOrCreate()


In [247]:
spark

# read dataset

In [248]:

df_pyspark = spark.read.csv(r'C:\Users\samah\Desktop\test.csv')

In [249]:
df_pyspark = spark.read.option('header','true').csv(r'C:\Users\samah\Desktop\test.csv',inferSchema=True) #.show()


In [250]:
df_pyspark.head(3)

[Row(name='samaha', departement='IOT', experience=1000),
 Row(name='imane', departement='Big data', experience=2000),
 Row(name='adam', departement='data science', experience=8)]

In [251]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [252]:
# check the schema (data types)
df_pyspark.printSchema()

root
 |-- name: string (nullable = true)
 |-- departement: string (nullable = true)
 |-- experience: integer (nullable = true)



# selecting columns and indexing 

In [253]:
df_pyspark.head()

Row(name='samaha', departement='IOT', experience=1000)

In [254]:
df_pyspark.select('name').show()

+-------+
|   name|
+-------+
| samaha|
|  imane|
|   adam|
|mohamed|
|  ahmed|
|   adam|
|   null|
|   null|
+-------+



In [255]:
df_pyspark.select(['name','departement']).show()

+-------+------------+
|   name| departement|
+-------+------------+
| samaha|         IOT|
|  imane|    Big data|
|   adam|data science|
|mohamed|         IOT|
|  ahmed|    Big data|
|   adam|data science|
|   null|        null|
|   null|          37|
+-------+------------+



In [256]:
df_pyspark['Name'] #we cant apply show function 


Column<'Name'>

In [257]:
df_pyspark.dtypes

[('name', 'string'), ('departement', 'string'), ('experience', 'int')]

In [258]:
df_pyspark.describe().show()

+-------+------+------------+-----------------+
|summary|  name| departement|       experience|
+-------+------+------------+-----------------+
|  count|     6|           7|                7|
|   mean|  null|        37.0|           3099.0|
| stddev|  null|        null|3342.268042312984|
|    min|  adam|          37|                6|
|    max|samaha|data science|             9000|
+-------+------+------------+-----------------+



In [259]:
### adding columns in data frame 
df_pyspark = df_pyspark.withColumn('Experience After 1 year',df_pyspark['experience']+2)

In [260]:
df_pyspark.show()

+-------+------------+----------+-----------------------+
|   name| departement|experience|Experience After 1 year|
+-------+------------+----------+-----------------------+
| samaha|         IOT|      1000|                   1002|
|  imane|    Big data|      2000|                   2002|
|   adam|data science|         8|                     10|
|mohamed|         IOT|      9000|                   9002|
|  ahmed|    Big data|      4000|                   4002|
|   adam|data science|      5679|                   5681|
|   null|        null|         6|                      8|
|   null|          37|      null|                   null|
+-------+------------+----------+-----------------------+



# drop columns 

In [261]:
df_pyspark = df_pyspark.drop('Experience After 1 year')


In [262]:
df_pyspark.show()

+-------+------------+----------+
|   name| departement|experience|
+-------+------------+----------+
| samaha|         IOT|      1000|
|  imane|    Big data|      2000|
|   adam|data science|         8|
|mohamed|         IOT|      9000|
|  ahmed|    Big data|      4000|
|   adam|data science|      5679|
|   null|        null|         6|
|   null|          37|      null|
+-------+------------+----------+



# rename the column

In [263]:
df_pyspark = df_pyspark.withColumnRenamed('name','Name')

In [264]:
df_pyspark.show()

+-------+------------+----------+
|   Name| departement|experience|
+-------+------------+----------+
| samaha|         IOT|      1000|
|  imane|    Big data|      2000|
|   adam|data science|         8|
|mohamed|         IOT|      9000|
|  ahmed|    Big data|      4000|
|   adam|data science|      5679|
|   null|        null|         6|
|   null|          37|      null|
+-------+------------+----------+



# handling missing values by mean / median / mode

In [265]:
df_pyspark = df_pyspark.na.drop()



In [266]:
# filling missing values
#df_pyspark.na.fill('Missing values',['experience','age','Name']).show()


In [267]:
from pyspark.ml.feature import Imputer


In [268]:
imputer = Imputer(
    inputCols=['experience'],
    outputCols=["{}_imputed".format(c) for c in ['experience']]
  

).setStrategy("mean")

In [269]:
imputer.fit(df_pyspark).transform(df_pyspark).show()

+-------+------------+----------+------------------+
|   Name| departement|experience|experience_imputed|
+-------+------------+----------+------------------+
| samaha|         IOT|      1000|              1000|
|  imane|    Big data|      2000|              2000|
|   adam|data science|         8|                 8|
|mohamed|         IOT|      9000|              9000|
|  ahmed|    Big data|      4000|              4000|
|   adam|data science|      5679|              5679|
+-------+------------+----------+------------------+



# filter operation 

In [271]:
 # age less than 25
df_pyspark.filter('experience<=2000').select(['Name']).show()

+------+
|  Name|
+------+
|samaha|
| imane|
|  adam|
+------+



In [273]:
df_pyspark.filter((df_pyspark['experience'] >1000) & (df_pyspark['experience'] <4000)).select(['Name']).show()

+-----+
| Name|
+-----+
|imane|
+-----+



# groupBy and aggregate function 

In [274]:
from pyspark.sql import SparkSession

In [279]:
df_pyspark.groupBy('departement').mean().show() # or sum() count() avg()

+------------+---------------+
| departement|avg(experience)|
+------------+---------------+
|         IOT|         5000.0|
|data science|         2843.5|
|    Big data|         3000.0|
+------------+---------------+



In [280]:
# group by dep that have higher salary 
df_pyspark.agg({'experience' : 'sum'}).show()


+---------------+
|sum(experience)|
+---------------+
|          21687|
+---------------+



In [ ]:
 #done